### Sequential Chain & Conditional Routing

##### Boilerplate code

In [2]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


## RunnableBranch - Conditional routing

In [5]:
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough


def data_for_super_admin(payload):
    return {
        "id": 1001,
        "name": "Rajesh Kumar",
        "email": "rajesh.kumar@company.com",
        "phone": "+91-9876543210",
        "role": "super_admin",
        "department": "IT Operations",
        "employee_id": "SA001",
        "permissions": ["all_access", "user_management", "system_config", "billing_access", "audit_logs", "role_management", "organization_management"],
        "access_level": 10,
        "salary": 2500000,
        "team_members": ["1002", "1003", "1004", "1005"],
        "projects": ["Infrastructure", "Security", "Compliance"],
        "certifications": ["AWS Certified", "CISSP", "PMP"],
        "created_at": "2023-01-15T09:30:00Z",
        "last_login": "2025-09-20T10:15:22Z",
        "login_count": 1247,
        "failed_attempts": 0,
        "session_timeout": 480,
        "status": "active"
    }

def data_for_admin(payload):
    return {
        "id": 1002,
        "name": "Priya Sharma", 
        "email": "priya.sharma@company.com",
        "role": "admin",
        "department": "Marketing",
        "permissions": ["user_management", "content_edit", "reports_view", "team_management"],
        "access_level": 7,
        "reporting_manager": "1001",
        "team_members": ["1003"],
        "created_at": "2023-03-22T14:45:00Z",
        "last_login": "2025-09-20T08:30:45Z",
        "status": "active"
    }

def data_for_user(payload):
    return {
        "id": 1003,
        "name": "Arjun Patel",
        "email": "arjun.patel@gmail.com", 
        "role": "user",
        "permissions": ["profile_edit", "content_view"],
        "created_at": "2024-06-10T11:20:00Z",
        "last_login": "2025-09-19T16:42:18Z",
        "status": "active"
    }



prompt_template = PromptTemplate.from_template("Simplify this data in clean table format for human consumption. Here is the data: {data}")

super_admin_chain = RunnablePassthrough.assign(data=RunnableLambda(data_for_super_admin)) | prompt_template | google_llm | StrOutputParser()
admin_chain = RunnablePassthrough.assign(data=RunnableLambda(data_for_admin)) | prompt_template | google_llm |  StrOutputParser()
user_chain = RunnablePassthrough.assign(data=RunnableLambda(data_for_user)) | prompt_template | google_llm | StrOutputParser()

role_router = RunnableBranch(
    (lambda x: x['role'] == 'super_admin', super_admin_chain),
    (lambda x: x['role'] == 'admin', admin_chain),
    user_chain,
)

role_router.invoke({"role": "test"})

"Okay, here's the simplified data in a table format:\n\n| Field         | Value                       |\n|---------------|-----------------------------|\n| ID            | 1003                        |\n| Name          | Arjun Patel                 |\n| Email         | arjun.patel@gmail.com       |\n| Role          | User                        |\n| Permissions   | profile_edit, content_view |\n| Created At    | 2024-06-10 11:20:00 UTC    |\n| Last Login    | 2025-09-19 16:42:18 UTC    |\n| Status        | Active                      |"